In [2]:
from transformers import pipeline

# Load the summarization pipeline

summarizer = pipeline("summarization")

# Input text to be summarized

text = """

Hugging Face is a company that specializes in natural language processing (NLP).

It has developed the Transformers library, which provides state-of-the-art models

for a wide range of NLP tasks such as text classification, information extraction,

question answering, summarization, translation, and more. The library is widely used

in both academia and industry due to its ease of use and flexibility.

"""

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [3]:
# Generate summary

summary = summarizer(text, max_length=50, min_length=20, do_sample=False)

# Display the summary

print("Summary:", summary[0]['summary_text'])

Summary:  The Transformers library provides state-of-the-art models for a wide range of NLP tasks such as text classification, information extraction, question answering, summarization, translation, and more .


In [4]:
!pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [17]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
# Load a pretrained model and tokenizer for topic classification
model_name = "cardiffnlp/tweet-topic-21-multi"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
# Define label mapping (from model documentation)
labels = [
    "arts_&culture", "business&entrepreneurs", "celebrity&pop_culture", "diaries&_daily_life",
    "family", "fashion_&style", "film_tv&video", "fitness&health", "food&_dining",
    "gaming", "learning_&educational", "music", "news&_social_concern", "other_hobbies",
    "relationships", "science_&technology", "sports&esports", "travel&_adventure",
    "youth_&_student_life"
]
# Example texts
texts = [
    "The latest iPhone was just released with an incredible new camera!",
    "Manchester United won their match with a stunning goal in the last minute.",
    "NASA just launched a new mission to explore the surface of Mars.",
    "The Oscars had some surprising winners this year!"
]
# Tokenize the input texts
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
# Perform classification
with torch.no_grad():
    outputs = model(**inputs)
# Convert logits to probabilities using softmax
probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions=torch.argmax(probabilities, dim=1)
# Print results
for text, pred, prob in zip(texts, predictions, probabilities):
    print(f"Text: {text}\nTopic: {labels[pred.item()]}, Confidence: {prob[pred].item():.4f}\n")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Text: The latest iPhone was just released with an incredible new camera!
Topic: science_&technology, Confidence: 0.9260

Text: Manchester United won their match with a stunning goal in the last minute.
Topic: sports&esports, Confidence: 0.9989

Text: NASA just launched a new mission to explore the surface of Mars.
Topic: science_&technology, Confidence: 0.8526

Text: The Oscars had some surprising winners this year!
Topic: film_tv&video, Confidence: 0.9357



In [16]:

from transformers import AutoModelForCausalLM, AutoTokenizer

# Load a pre-trained text generation model and tokenizer

model_name = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(model_name)

# Define a prompt

prompt = "Once upon a time there lived a ghost,"

# Encode the input text

inputs = tokenizer(prompt, return_tensors="pt")

# Generate text

output = model.generate(**inputs, max_length=50, num_return_sequences=1, temperature=0.7, top_k=50)

# Decode and print the generated text

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time there lived a ghost, and it was not a man, but a woman, who had been slain by the devil.

The devil, who had been slain by the devil, was the first to come to the earth,


In [18]:
import os

import atexit

import shutil

from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration

# Step 1: Load the pre-trained BlenderBot model and tokenizer

model_name = "facebook/blenderbot-1B-distill"

tokenizer = BlenderbotTokenizer.from_pretrained(model_name)

model = BlenderbotForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/127k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/62.9k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/310k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.87G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.87G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]